In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import gzip

import numpy as np
from IPython import display

import time

from keras import backend as K
from keras.callbacks import Callback
from keras.models import Model, load_model, model_from_json
from keras.optimizers import SGD, Adam
from keras.layers import Dense, Dropout, GlobalAveragePooling2D # Conv2D, Input, Flatten, MaxPooling2D, UpSampling2D, concatenate, Cropping2D, Reshape, BatchNormalization
from keras.utils import HDF5Matrix
from keras.applications.vgg16 import VGG16

from keras.models import load_model
import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow as tf

Using TensorFlow backend.


In [5]:
import sys
sys.path.append("..")

In [8]:
from utils.load_data import load_data
from utils.preprocess import DataGenerator
from utils.comparams import calculate_auc, auc, show

### Data Generator

In [12]:
data_dir = '../data'
# data_dir = './data'

In [13]:
x_test, y_test_true = load_data(data_dir, purpose='test')#, norm='macenko')

In [14]:
# indexes
test_id = np.arange(len(x_test))

# create a useful dictionary structures
partition = {}
partition['test'] = test_id
    
test_labels = {str(i) : y_test_true[i].flatten()[0] for i in test_id}

In [10]:
len(test_labels)

32768

## VGG16 Predictions

In [21]:
# directory to save the best model
file_dir = '../Model'
pred_file_dir = '../Preds'
model_name = 'vgg16_model_100K'    
network_filepath = os.path.join(file_dir, model_name + '.h5') #_limitless

In [22]:
# Define batch size.
batch_size = 128

# Parameters for generators
params = {
    'dim': (224, 224),
    'batch_size': batch_size,
    'n_classes': 2,
    'shuffle': False
}

# Generators
test_generator = DataGenerator(partition['test'], x_test, test_labels, **params)

In [25]:
dependencies = {
    'auc': auc
}

In [26]:
vgg16_model = load_model(network_filepath, custom_objects=dependencies)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Instructions for updating:
Use tf.cast instead.


In [ ]:
vgg16_preds = vgg16_model.predict_generator(test_generator)

# VGG19

In [ ]:
model_name_vgg19 = 'vgg19_model_10K_lrr_0-001'    
network_filepath_vgg19 = os.path.join(file_dir, model_name_vgg19 + '.h5') 
vgg19_model = load_model(network_filepath_vgg19, custom_objects=dependencies)

In [ ]:
vgg19_preds = vgg19_model.predict_generator(test_generator)

# NASNET

In [ ]:
# directory to save the best model
file_dir = '../Model/weights'
model_name_nasnet = 'nasnet_modelbest_acc_model'
network_filepath_nasnet = os.path.join(file_dir, model_name_nasnet + '.h5') #_limitless


In [ ]:
nasnet = NASNetLarge(weights='imagenet', include_top=False)

In [ ]:
inputs = Input((224, 224, 3))
x = nasnet(inputs)
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
y = Dense(2, activation='softmax')(x) # sigmoid instead of softmax to have independent probabilities


In [ ]:
for layer in nasnet.layers:
    layer.trainable = False
nasnet_model = Model(inputs=inputs, outputs=y)

In [ ]:
nasnet_model.load_weights(network_filepath_nasnet)

In [ ]:
nasnet_preds = nasnet_model.predict_generator(test_generator)